## Preprocessing

In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

2023-11-14 19:16:01.099755: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'], axis=1)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [5]:
# Look at APPLICATION_TYPE value counts for binning
app_count = application_df['APPLICATION_TYPE'].value_counts()
app_count

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(app_count[app_count < 500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts for binning
class_count = application_df['CLASSIFICATION'].value_counts()
class_count

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [8]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_count_1 = class_count.loc[class_count > 1]
class_count_1.head()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [9]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(class_count[class_count < 1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [10]:
# Convert categorical data to numeric with `pd.get_dummies`
numeric = pd.get_dummies(application_df)

In [14]:
# Split our preprocessed data into our features and target arrays
x = numeric.drop(['IS_SUCCESSFUL'], axis=1)
y= numeric['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
x_train, x_test, y_train, y_test = train_test_split(x,y, random_state=1)

In [16]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
x_scaler = scaler.fit(x_train)

# Scale the data
x_train_scaled = x_scaler.transform(x_train)
x_test_scaled = x_scaler.transform(x_test)

## Compile, Train and Evaluate the Model

In [17]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# attempt 1:
input_features = len(x_train_scaled[0])
layer1 = 10
layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=layer1, input_dim=input_features, activation='tanh'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=layer2, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='relu'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                440       
                                                                 
 dense_1 (Dense)             (None, 5)                 55        
                                                                 
 dense_2 (Dense)             (None, 1)                 6         
                                                                 
Total params: 501 (1.96 KB)
Trainable params: 501 (1.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [20]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [29]:
# Train the model
fit = nn.fit(x_train_scaled, y_train, epochs=20)

Epoch 1/20
804/804 [==============================] - 2s 3ms/step - loss: 0.5476 - accuracy: 0.7326
Epoch 2/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5475 - accuracy: 0.7336
Epoch 3/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5485 - accuracy: 0.7334
Epoch 4/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5499 - accuracy: 0.7333
Epoch 5/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5480 - accuracy: 0.7336
Epoch 6/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5461 - accuracy: 0.7334
Epoch 7/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5501 - accuracy: 0.7329
Epoch 8/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5461 - accuracy: 0.7340
Epoch 9/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5471 - accuracy: 0.7337
Epoch 10/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5465 - accuracy: 0.7338

In [30]:
# Evaluate the model using the test data 
model_loss, model_accuracy = nn.evaluate(x_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5606 - accuracy: 0.7292 - 335ms/epoch - 1ms/step
Loss: 0.5605982542037964, Accuracy: 0.7292128205299377


In [31]:
# attempt 2:
input_features = len(x_train_scaled[0])
layer1 = 10
layer2 = 10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=layer1, input_dim=input_features, activation='tanh'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=layer2, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='relu'))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 10)                440       
                                                                 
 dense_4 (Dense)             (None, 10)                110       
                                                                 
 dense_5 (Dense)             (None, 1)                 11        
                                                                 
Total params: 561 (2.19 KB)
Trainable params: 561 (2.19 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [32]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [33]:
# Train the model (try #2)
fit = nn.fit(x_train_scaled, y_train, epochs=20)

Epoch 1/20
804/804 [==============================] - 3s 2ms/step - loss: 3.7040 - accuracy: 0.5420
Epoch 2/20
804/804 [==============================] - 2s 2ms/step - loss: 3.3671 - accuracy: 0.6048
Epoch 3/20
804/804 [==============================] - 2s 2ms/step - loss: 3.2759 - accuracy: 0.6070
Epoch 4/20
804/804 [==============================] - 1s 2ms/step - loss: 2.9058 - accuracy: 0.6160
Epoch 5/20
804/804 [==============================] - 1s 2ms/step - loss: 2.0066 - accuracy: 0.6433
Epoch 6/20
804/804 [==============================] - 2s 2ms/step - loss: 1.0789 - accuracy: 0.6732
Epoch 7/20
804/804 [==============================] - 2s 2ms/step - loss: 0.8423 - accuracy: 0.6999
Epoch 8/20
804/804 [==============================] - 2s 2ms/step - loss: 0.6839 - accuracy: 0.6177
Epoch 9/20
804/804 [==============================] - 1s 2ms/step - loss: 0.6232 - accuracy: 0.7033
Epoch 10/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5994 - accuracy: 0.7114

In [34]:
# Evaluate the model using the test data 
model_loss, model_accuracy = nn.evaluate(x_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5776 - accuracy: 0.7242 - 620ms/epoch - 2ms/step
Loss: 0.5776026248931885, Accuracy: 0.7241982221603394


In [35]:
# Attempt 3:
input_features = len(x_train_scaled[0])
layer1 = 20
layer2 = 20
layer3 = 20

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=layer1, input_dim=input_features, activation='tanh'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=layer2, activation='relu'))

# third hidden layer
nn.add(tf.keras.layers.Dense(units=layer3, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 20)                880       
                                                                 
 dense_7 (Dense)             (None, 20)                420       
                                                                 
 dense_8 (Dense)             (None, 20)                420       
                                                                 
 dense_9 (Dense)             (None, 1)                 21        
                                                                 
Total params: 1741 (6.80 KB)
Trainable params: 1741 (6.80 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [36]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [38]:
# Train the model (try #3)
fit = nn.fit(x_train_scaled, y_train, epochs=50)

Epoch 1/50
804/804 [==============================] - 3s 3ms/step - loss: 0.5425 - accuracy: 0.7362
Epoch 2/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5428 - accuracy: 0.7355
Epoch 3/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5423 - accuracy: 0.7360
Epoch 4/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5420 - accuracy: 0.7365
Epoch 5/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5420 - accuracy: 0.7364
Epoch 6/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5420 - accuracy: 0.7366
Epoch 7/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5420 - accuracy: 0.7369
Epoch 8/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5416 - accuracy: 0.7369
Epoch 9/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5416 - accuracy: 0.7369
Epoch 10/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5415 - accuracy: 0.7374

In [39]:
# Evaluate the model using the test data 
model_loss, model_accuracy = nn.evaluate(x_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5541 - accuracy: 0.7303 - 443ms/epoch - 2ms/step
Loss: 0.5541417598724365, Accuracy: 0.7302623987197876


In [41]:
# export model:
filepath = '/Users/tincho/Desktop/Challenge Reps/deep-learning-challenge/AlphabetSoupCharity_Optimization.h5'
nn.save(filepath)


/Users/tincho/anaconda3/envs/dev/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
